# ГЛАВА 7

Создание модели привычным способом и с помощью API

In [ ]:
# keras API позволяет напрямую манипулировать тензорами и 
# рассматривать слои как функции
from keras.models import Sequential, Model
from keras.layers import Dense
from keras import Input
import numpy as np

# Привычный способ
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(64,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Эквивалент через API
# создаем входной тензор
input_tensor = Input(shape=(64,))
# в первый слой передаем входной тензор
x = Dense(128, activation='relu')(input_tensor)
# во второй слой передаем первый слой
x = Dense(64, activation='relu')(x)
# в каждый последующий слой передаем предыдущий
output_tensor = Dense(10, activation='softmax')(x)

# создаем модель, указывая входной и выходной тензоры
model = Model(input_tensor, output_tensor)

model.summary()

# компиляция, обучение, оценка - такие же
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['mae']
)

x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=20
)

score = model.evaluate(x_train, y_train)


Создание модели типа "вопрос-ответ" с несколькими входами

In [8]:
# обычно модель "вопрос-ответ" получает на вход вопрос на естественном языке
# и текст, на основе которого будет даваться ответ

from keras.models import Model
from keras import layers
from keras import Input

text_size = 10_000
question_size = 10_000
answer_size = 500

# входной текст - последовательность целых чисел переменной длины
# форма - (количество образцов, длина образца)
text_input = Input(shape=(None,), dtype='int32', name='text')
# преобразование текста в последовательность векторов размером 64
embedded_text = layers.Embedding(text_size, 64)(text_input)
# преобразование в единый вектор
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='float32', name='question')
embedded_question = layers.Embedding(question_size, 64)(question_input)
encoded_question = layers.LSTM(32)(embedded_question)

# объединение закодированного вопроса и ответа
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# создание классификатора
answer = layers.Dense(answer_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# ПЕРЕДАЧА ДАННЫХ В МОДЕЛЬ С НЕСКОЛЬКИМИ ВХОДАМИ
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_size, size=(num_samples, max_length))
question = np.random.randint(1, question_size, size=(num_samples, max_length))
answers = np.zeros(shape=(num_samples, answer_size))
# массив индексов
indices = np.random.randint(0, answer_size, size=num_samples)
# прямое кодирование вопросов
for i, x in enumerate(answers):
  x[indices[i]] = 1

# можно передать так
model.fit([text, question], answers, epochs=20, batch_size=128)
# но если при создании через Input тензорам давались имена, то можно передать и через словарь
model.fit({'text':text, 'question':question}, answers, epochs=20, batch_size=128)

model.evaluate([text, question], answers)

Epoch 1/20
8/8 [==============================] - 0s 26ms/step - loss: 6.2145 - accuracy: 0.0020
Epoch 2/20
8/8 [==============================] - 0s 26ms/step - loss: 6.1890 - accuracy: 0.0330
Epoch 3/20
8/8 [==============================] - 0s 25ms/step - loss: 6.1026 - accuracy: 0.0040
Epoch 4/20
8/8 [==============================] - 0s 25ms/step - loss: 6.0131 - accuracy: 0.0030
Epoch 5/20
8/8 [==============================] - 0s 26ms/step - loss: 5.9218 - accuracy: 0.0050
Epoch 6/20
8/8 [==============================] - 0s 25ms/step - loss: 5.8282 - accuracy: 0.0130
Epoch 7/20
8/8 [==============================] - 0s 27ms/step - loss: 5.7401 - accuracy: 0.0230
Epoch 8/20
8/8 [==============================] - 0s 25ms/step - loss: 5.6365 - accuracy: 0.0270
Epoch 9/20
8/8 [==============================] - 0s 24ms/step - loss: 5.5529 - accuracy: 0.0370
Epoch 10/20
8/8 [==============================] - 0s 25ms/step - loss: 5.4571 - accuracy: 0.0290
Epoch 11/20
8/8 [============

[3.438575267791748, 0.5130000114440918]